### Creates a dictionary of crop types from [CropScape](https://nassgeodata.gmu.edu/CropScape/]) data in radar coords and masks based on if crop type changed between 2019-2021

In [1]:
# all the good stuff
import os,sys
import glob
import pandas as pd
import json
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# create list of all Cropscape radar files
workdir = '/data/krd86/cropped/cropscape/'
years = [os.path.basename(x) for x in glob.glob(workdir+"radar/CDL*")]
years = sorted(years)
ny = len(years)

# size of downlooked file
dx = 1500
dy = 875

In [3]:
f = open(workdir+'dict/cropflag.txt')
cropflag = [int(x) for x in f.read().splitlines()]
f.close()

f = open(workdir+'dict/cropname.txt')
cropname = f.read().splitlines()
f.close()

In [4]:
crop_dict = []
for i, value in enumerate(cropflag):
    flag = value
    label = cropname[i]
    crop_dict.append([flag,label])

crop_dict = pd.DataFrame(crop_dict,columns=['flag','label'])
crop_dict.to_json(workdir+"dict/crop_dict.json")

In [5]:
crop_dict.label[0]

'Corn'

In [14]:
# create mask of crops 2019-2021
f1 = gdal.Open(workdir+'original/CDL_2019_clip_20220408094633_39327711.tif', gdal.GA_ReadOnly)
crop_2019 = f1.ReadAsArray()
f2 = gdal.Open(workdir+'original/CDL_2020_clip_20220408094633_39327711.tif', gdal.GA_ReadOnly)
crop_2020 = f2.ReadAsArray()

same_mask = np.ones([dy,dx])
same_mask[crop_2020!=crop_2019] = np.nan
diff_mask = np.ones([dy,dx])
diff_mask[crop_2020==crop_2019] = np.nan

driver=gdal.GetDriverByName('ISCE')
mfile = driver.Create(workdir+'dict/same_mask.r4',1500,875,1,gdal.GDT_Float32)
mfile.GetRasterBand(1).WriteArray(same_mask)
mfile=None

mfile = driver.Create(workdir+'dict/diff_mask.r4',1500,875,1,gdal.GDT_Float32)
mfile.GetRasterBand(1).WriteArray(diff_mask)
mfile=None